In [1]:
import numpy as np
import pandas as pd

In [2]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [3]:
phenotypes_data=pd.read_csv('agrocodehub_data/phenotypes.tsv',sep='\t')
vegetation_data=pd.read_csv('agrocodehub_data/vegetation.tsv',sep='\t')

In [4]:
phenotypes_data.head()

,sample,2015,2016,2017,2019,2020,2021,2022,2023
0,PS000196,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN
1,PS000195,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN
2,PS000121,NaN,NaN,101.0,107.0,NaN,NaN,NaN,NaN
3,PS000126,NaN,NaN,NaN,115.0,112.0,NaN,NaN,NaN
4,PS000123,NaN,NaN,NaN,113.0,102.0,79.0,NaN,NaN


In [5]:
vegetation_data.head()

,sample,vegetation
0,PS000196,100
1,PS000195,103
2,PS000121,104
3,PS000126,106
4,PS000123,101


In [6]:
phenotypes_data

,sample,2015,2016,2017,2019,2020,2021,2022,2023
0,PS000196,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN
1,PS000195,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN
2,PS000121,NaN,NaN,101.0,107.0,NaN,NaN,NaN,NaN
3,PS000126,NaN,NaN,NaN,115.0,112.0,NaN,NaN,NaN
4,PS000123,NaN,NaN,NaN,113.0,102.0,79.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
94,PS000370,NaN,NaN,NaN,NaN,NaN,128.0,85.0,NaN
95,PS000184,101.0,98.0,95.0,95.0,79.0,95.0,NaN,NaN
96,PS000169,122.0,NaN,121.0,120.0,151.0,NaN,NaN,NaN
97,PS000053,NaN,NaN,105.0,98.0,NaN,NaN,NaN,NaN


In [7]:
vegetation_data

,sample,vegetation
0,PS000196,100
1,PS000195,103
2,PS000121,104
3,PS000126,106
4,PS000123,101
...,...,...
94,PS000370,118
95,PS000184,102
96,PS000169,114
97,PS000053,119


In [8]:
set(vegetation_data["sample"].unique()) - set(phenotypes_data["sample"].unique())

set()

In [9]:
data = pd.merge(phenotypes_data, vegetation_data, on=["sample"])
data

,sample,2015,2016,2017,2019,2020,2021,2022,2023,vegetation
0,PS000196,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN,100
1,PS000195,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,103
2,PS000121,NaN,NaN,101.0,107.0,NaN,NaN,NaN,NaN,104
3,PS000126,NaN,NaN,NaN,115.0,112.0,NaN,NaN,NaN,106
4,PS000123,NaN,NaN,NaN,113.0,102.0,79.0,NaN,NaN,101
...,...,...,...,...,...,...,...,...,...,...
94,PS000370,NaN,NaN,NaN,NaN,NaN,128.0,85.0,NaN,118
95,PS000184,101.0,98.0,95.0,95.0,79.0,95.0,NaN,NaN,102
96,PS000169,122.0,NaN,121.0,120.0,151.0,NaN,NaN,NaN,114
97,PS000053,NaN,NaN,105.0,98.0,NaN,NaN,NaN,NaN,119


In [10]:
data.describe()

,2015,2016,2017,2019,2020,2021,2022,2023,vegetation
count,43.000000,34.000000,39.000000,81.000000,67.000000,61.000000,19.000000,15.000000,99.000000
mean,106.790698,99.911765,100.615385,101.679012,105.164179,108.721311,104.368421,104.533333,107.717172
std,15.905312,14.924797,13.548384,16.977800,17.716280,14.740117,10.414789,14.730273,5.562338
min,71.000000,69.000000,73.000000,72.000000,59.000000,67.000000,84.000000,69.000000,93.000000
25%,99.500000,92.000000,94.500000,93.000000,94.500000,102.000000,99.000000,97.500000,104.000000
50%,108.000000,102.000000,102.000000,99.000000,106.000000,108.000000,106.000000,107.000000,108.000000
75%,116.500000,112.000000,106.500000,107.000000,116.000000,119.000000,113.500000,113.500000,112.000000
max,140.000000,123.000000,124.000000,180.000000,151.000000,141.000000,116.000000,130.000000,127.000000


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sample      99 non-null     object 
 1   2015        43 non-null     float64
 2   2016        34 non-null     float64
 3   2017        39 non-null     float64
 4   2019        81 non-null     float64
 5   2020        67 non-null     float64
 6   2021        61 non-null     float64
 7   2022        19 non-null     float64
 8   2023        15 non-null     float64
 9   vegetation  99 non-null     int64  
dtypes: float64(8), int64(1), object(1)
memory usage: 7.9+ KB


In [12]:
weather_features = pd.read_csv("Weather features.csv").drop(index=[0,1]).rename(columns={"Unnamed: 0": "year"})
weather_features["year"] = weather_features["year"].astype(int)
weather_features

,year,Т(С),Т(С).1,Т(С).2,Т(С).3,Т(С).4,Тd(С),Тd(С).1,Тd(С).2,Тd(С).3,...,P(гПа).4,Po(гПа),Po(гПа).1,Po(гПа).2,Po(гПа).3,Po(гПа).4,T_critical_days,T_nogrowth_days,T_middling_days,T_optimal_days
2,2015,16.314113,21.207917,20.712903,19.155645,17.365000,6.395565,12.598750,13.577016,9.640726,...,1019.957083,993.711694,994.912083,993.306855,1000.552016,1001.644167,4.0,153.0,74.0,69.0
3,2016,15.308065,19.803750,22.849194,22.140323,12.684167,9.516129,12.635000,13.895968,14.306452,...,1013.996250,994.500403,996.538333,993.798387,997.894758,995.515833,6.0,153.0,77.0,63.0
4,2017,13.699597,17.378750,20.678629,21.430364,14.940000,4.881452,9.879167,12.282661,12.314919,...,1020.126250,995.589919,991.655000,993.020161,998.868952,1001.658750,4.0,152.0,68.0,68.0
5,2019,16.822177,21.674583,18.652419,19.162500,14.319167,10.193145,12.939583,11.627016,10.886694,...,1016.910833,984.478629,988.383750,980.197984,987.036290,987.538333,1.0,153.0,66.0,77.0
6,2020,11.643145,20.985000,20.670565,19.255870,16.767083,5.518548,12.094167,10.802016,9.393522,...,1018.748333,982.829839,984.382083,984.293145,985.496761,989.552917,2.0,152.0,71.0,70.0
7,2021,14.677419,20.178750,23.121371,21.951210,11.343460,6.930645,13.018750,14.190323,12.776210,...,1016.192405,983.374194,985.202917,984.912097,984.665323,986.634599,6.0,152.0,69.0,65.0
8,2022,12.164113,20.039167,19.782661,22.075000,10.915063,2.420161,12.036250,13.608065,13.455242,...,1012.734728,984.718952,985.620417,984.626210,988.247177,983.189121,1.0,152.0,75.0,66.0
9,2023,14.301210,17.400000,19.855645,21.335081,16.441250,4.266129,9.677917,14.416532,15.326613,...,1022.355417,991.049194,985.437083,981.608065,986.568548,993.235417,1.0,153.0,60.0,86.0


In [13]:
!pip install bioinfokit

Defaulting to user installation because normal site-packages is not writeable


In [14]:
from bioinfokit.analys import marker

# id is for column name with chromosome identifier in VCF file
vcf_iter = marker.vcfreader(file='agrocodehub_data/genotypes.vcf', id='#CHROM')

header_list = []
info_lines_list = []
marker_lines_list = []

# read vcf file
for record in vcf_iter:
    # get info lines, header, and variant records
    headers, info_lines, marker_lines = record

    header_list.append(headers)
    info_lines_list.append(info_lines)
    marker_lines_list.append(marker_lines)

In [15]:
header_list = header_list[0]
info_lines_list = info_lines_list[0]

In [16]:
info_lines_list[-24:]

['##contig=<ID=KZ848287,length=1002>',
 '##ALT=<ID=*,Description="Represents allele(s) other than observed.">',
 '##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">',
 '##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of raw reads supporting an indel">',
 '##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of raw reads supporting an indel">',
 '##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">',
 '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">',
 '##INFO=<ID=RPB,Number=1,Type=Float,Description="Mann-Whitney U test of Read Position Bias (bigger is better)">',
 '##INFO=<ID=MQB,Number=1,Type=Float,Description="Mann-Whitney U test of Mapping Quality Bias (bigger is better)">',
 '##INFO=<ID=BQB,Number=1,Type=Float,Description="Mann-Whitney U test of Base Quality Bias (bigger is better)">',
 '##INFO=<

In [17]:
# Создание словаря
contig_dict = {}

for contig in info_lines_list[3:-23]:
    # Извлечение ID и length
    id_part = contig.split(',')[0].split('=')[2]  # Получаем ID
    length_part = contig.split(',')[1].split('=')[1][:-1]  # Получаем length
    contig_dict[id_part] = int(length_part)  # Добавляем в словарь

In [18]:
genotypes_data = pd.DataFrame(data=marker_lines_list, columns=header_list)
genotypes_data

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,PS000026,...,PS000433,PS000436,PS000437,PS000440,PS000441,PS000444,PS000566,PS000567,PS000568,PS000570
0,1,40481,.,G,A,132,.,.,GT:PL:DP,"0/0:0,3,39:1",...,"0/0:0,3,48:1","./.:0,0,0:0","0/0:0,3,60:1","0/0:0,3,60:1","0/0:0,9,151:3","0/0:0,27,220:9","0/0:0,66,250:22","0/0:0,47,247:20","0/0:0,99,251:33","0/0:0,60,250:20"
1,1,40522,.,C,T,999,.,.,GT:PL:DP,"0/0:0,3,40:1",...,"0/0:0,3,25:1","./.:0,0,0:0","0/0:0,3,37:1","0/0:0,3,37:1","0/0:0,9,92:3","0/0:0,27,170:9","0/0:0,66,196:22","0/0:0,47,193:20","0/0:0,99,199:33","0/0:0,57,195:19"
2,1,111641,.,G,C,999,.,.,GT:PL:DP,"0/1:39,3,0:1",...,"./.:0,0,0:0","0/1:48,3,0:1","0/1:255,0,24:12","0/1:191,0,42:6","0/1:137,0,96:5","0/0:0,15,212:5","0/1:255,0,223:56","0/0:0,21,254:7","0/1:255,0,194:37","0/0:0,39,255:13"
3,1,111675,.,G,C,999,.,.,GT:PL:DP,"0/1:49,3,0:1",...,"./.:0,0,0:0","0/1:60,3,0:1","0/1:255,0,24:12","0/1:197,0,42:6","0/1:137,0,96:5","0/0:0,15,212:5","0/1:255,0,221:56","0/0:0,21,254:7","0/1:255,0,229:38","0/0:0,39,255:13"
4,1,111688,.,T,C,999,.,.,GT:PL:DP,"0/1:40,3,0:1",...,"./.:0,0,0:0","0/1:60,3,0:1","0/1:255,0,24:12","0/1:197,0,42:6","0/1:137,0,96:5","0/0:0,15,212:5","0/1:255,0,223:56","0/0:0,21,254:7","0/1:255,0,226:38","0/0:0,39,255:13"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38935,KZ848124,2425,.,T,A,999,.,.,GT:PL:DP,"0/1:32,0,123:7",...,"0/1:55,0,151:14","0/1:103,0,131:12","0/1:166,0,151:42","0/1:100,0,129:13","0/1:166,0,146:34","0/0:0,33,41:11","0/1:153,0,87:37","0/1:165,0,143:80","0/1:158,0,163:51","0/1:161,0,191:83"
38936,KZ848124,2494,.,T,C,999,.,.,GT:PL:DP,"0/1:35,0,159:7",...,"0/1:58,0,252:14","0/1:110,0,205:12","0/1:232,0,255:42","0/1:127,0,193:12","0/1:178,0,255:35","./.:0,0,0:0","0/1:175,0,255:103","0/1:222,0,255:121","0/1:132,0,255:76","0/1:176,0,255:134"
38937,KZ848124,2502,.,T,A,999,.,.,GT:PL:DP,"0/0:0,21,150:7",...,"0/0:0,42,255:14","0/1:51,0,205:12","0/0:0,126,255:42","0/1:59,0,207:12","0/0:0,105,255:35","0/0:0,6,8:2","0/0:0,255,255:103","0/0:0,255,255:122","0/0:0,32,255:77","0/0:0,255,255:134"
38938,KZ848124,2515,.,G,T,999,.,.,GT:PL:DP,"0/0:0,21,149:7",...,"0/1:27,0,239:14","0/0:0,36,255:12","0/0:0,32,255:42","0/1:59,0,209:12","0/0:0,57,255:35","0/0:0,6,9:2","0/0:0,54,255:103","0/1:41,0,250:122","0/0:0,14,255:76","0/1:56,0,255:134"


In [19]:
genotypes_data.drop(columns=["ID", "FILTER", "INFO", "FORMAT"], inplace=True)

In [20]:
genotypes_data.iloc[:, :5].describe()

,#CHROM,POS,REF,ALT,QUAL
count,38940,38940,38940,38940,38940
unique,110,38897,1429,1489,1019
top,18,10146005,G,T,999
freq,2740,3,9554,10150,35337


In [21]:
genotypes_data.head()

,#CHROM,POS,REF,ALT,QUAL,PS000026,PS000027,PS000028,PS000031,PS000033,...,PS000433,PS000436,PS000437,PS000440,PS000441,PS000444,PS000566,PS000567,PS000568,PS000570
0,1,40481,G,A,132,"0/0:0,3,39:1","0/0:0,9,89:3","0/0:3,0,120:8","0/0:0,39,203:13","0/1:10,0,115:11",...,"0/0:0,3,48:1","./.:0,0,0:0","0/0:0,3,60:1","0/0:0,3,60:1","0/0:0,9,151:3","0/0:0,27,220:9","0/0:0,66,250:22","0/0:0,47,247:20","0/0:0,99,251:33","0/0:0,60,250:20"
1,1,40522,C,T,999,"0/0:0,3,40:1","0/1:21,0,85:4","0/1:115,0,172:22","0/0:0,57,212:19","0/1:149,0,172:31",...,"0/0:0,3,25:1","./.:0,0,0:0","0/0:0,3,37:1","0/0:0,3,37:1","0/0:0,9,92:3","0/0:0,27,170:9","0/0:0,66,196:22","0/0:0,47,193:20","0/0:0,99,199:33","0/0:0,57,195:19"
2,1,111641,G,C,999,"0/1:39,3,0:1","1/1:122,12,0:4","0/1:52,0,123:7","0/0:0,24,176:8","0/0:0,9,97:3",...,"./.:0,0,0:0","0/1:48,3,0:1","0/1:255,0,24:12","0/1:191,0,42:6","0/1:137,0,96:5","0/0:0,15,212:5","0/1:255,0,223:56","0/0:0,21,254:7","0/1:255,0,194:37","0/0:0,39,255:13"
3,1,111675,G,C,999,"0/1:49,3,0:1","1/1:150,12,0:4","0/1:64,0,186:7","0/0:0,24,255:8","0/0:0,9,151:3",...,"./.:0,0,0:0","0/1:60,3,0:1","0/1:255,0,24:12","0/1:197,0,42:6","0/1:137,0,96:5","0/0:0,15,212:5","0/1:255,0,221:56","0/0:0,21,254:7","0/1:255,0,229:38","0/0:0,39,255:13"
4,1,111688,T,C,999,"0/1:40,3,0:1","1/1:118,12,0:4","0/1:68,0,195:7","0/0:0,24,223:8","0/0:0,9,131:3",...,"./.:0,0,0:0","0/1:60,3,0:1","0/1:255,0,24:12","0/1:197,0,42:6","0/1:137,0,96:5","0/0:0,15,212:5","0/1:255,0,223:56","0/0:0,21,254:7","0/1:255,0,226:38","0/0:0,39,255:13"


In [22]:
genotypes_data["PS000433"].str.split(":").str[0].unique()

array(['0/0', './.', '0/1', '1/1', '1/2', '0/2', '1/3', '2/2', '0/3'],
      dtype=object)

In [23]:
genotypes_data[genotypes_data["QUAL"].astype(float) < 100]

,#CHROM,POS,REF,ALT,QUAL,PS000026,PS000027,PS000028,PS000031,PS000033,...,PS000433,PS000436,PS000437,PS000440,PS000441,PS000444,PS000566,PS000567,PS000568,PS000570
149,1,3599872,CAAAAAAAA,CAAAAAAAAA,80,"0/0:0,3,12:1","0/0:0,24,52:8","0/0:0,114,62:38","0/0:0,78,68:27","1/1:36,23,0:19",...,"0/0:0,12,27:4","./.:0,0,0:0","0/0:0,96,56:32","0/0:0,18,36:6","0/0:0,90,57:30","0/0:0,187,47:62","./.:0,0,0:0","0/0:0,175,48:58","0/0:24,24,11:128","0/0:0,255,42:114"
298,1,8074821,CTCT,C,39.1211,"0/0:0,6,47:2","0/0:0,48,200:16","0/0:0,78,227:26","0/0:3,0,86:20","0/0:0,78,254:26",...,"0/0:0,27,76:9","0/0:0,6,35:2","0/0:0,7,35:8","0/0:0,33,163:11","0/0:0,60,152:20","0/1:9,0,50:8","0/0:0,111,159:38","0/0:0,99,203:34","0/0:0,102,215:34","0/0:0,22,113:17"
317,1,11292121,T,A,51,"./.:0,0,0:0","0/0:4,0,28:4","0/0:0,24,96:8","0/0:0,5,56:7","0/0:0,21,106:7",...,"./.:0,0,0:0","./.:0,0,0:0","./.:0,0,0:0","./.:0,0,0:0","./.:0,0,0:0","0/0:0,36,182:12","0/0:0,45,187:15","0/0:0,36,182:12","0/0:0,27,163:9","0/0:0,24,159:8"
326,1,13209987,A,C,91,"0/0:0,6,7:2","0/0:0,51,9:17","0/0:0,33,12:11","./.:0,0,0:0","0/0:0,75,7:25",...,"0/0:0,9,9:3","0/0:0,3,4:1","0/0:0,33,12:11","0/0:0,6,7:2","0/0:0,12,11:4","0/0:0,9,9:3","0/0:0,72,7:24","0/0:0,78,7:26","./.:0,0,0:0","./.:0,0,0:0"
442,1,18327660,G,A,50,"./.:0,0,0:0","./.:0,0,0:0","0/1:12,3,0:1","0/0:0,3,40:1","0/1:2,0,90:10",...,"./.:0,0,0:0","0/0:0,3,60:1","0/0:0,3,60:1","0/0:0,3,60:1","0/0:0,3,60:1","0/0:0,12,176:4","0/0:0,15,212:5","0/0:0,18,205:6","0/0:0,15,255:11","0/0:0,24,255:8"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38346,KZ847136,567123,C,T,99,"0/1:0,3,4:1","0/1:0,3,9:1","0/0:0,6,7:2","0/0:0,18,20:6","0/0:0,5,6:11",...,"./.:0,0,0:0","./.:0,0,0:0","./.:0,0,0:0","0/1:0,3,4:1","0/1:0,1,7:2","0/1:0,3,9:1","0/1:0,3,4:1","0/1:0,3,4:1","0/0:0,15,58:5","0/1:0,3,4:1"
38359,KZ847139,286923,A,G,39.7275,"0/1:0,1,0:2","0/0:0,9,9:3","1/1:7,30,0:17","1/1:8,66,0:22","0/0:0,75,7:25",...,"1/1:6,32,0:21","1/1:1,6,0:16","1/1:1,23,0:84","1/1:2,32,0:58","1/1:3,43,0:100","0/0:0,81,5:32","1/1:9,54,0:18","0/0:0,105,6:35","0/0:0,216,9:78","1/1:1,17,0:69"
38670,KZ847789,1062,A,C,98,"0/0:0,6,60:2","0/1:8,0,40:8","0/0:0,23,102:25","0/0:0,35,117:19","0/0:0,23,161:21",...,"0/0:0,3,123:9","0/0:0,5,22:4","0/0:0,5,255:28","0/0:0,4,208:13","0/0:0,29,255:42","0/0:0,71,255:150","0/0:0,65,255:54","0/0:0,79,255:61","0/0:0,68,255:39","0/0:0,171,255:100"
38836,KZ847539,1292,C,G,43.6996,"1/1:9,9,0:3","0/1:0,1,0:2","1/1:9,9,0:3","0/0:0,4,2:7","0/1:0,3,3:3",...,"./.:0,0,0:0","0/1:0,3,4:1","0/1:0,3,3:3","1/1:4,3,0:1","1/1:4,3,0:1","./.:0,0,0:0","./.:0,0,0:0","1/1:7,6,0:2","1/1:10,48,0:16","1/1:8,16,0:11"


In [24]:
def set_nans(min_experiments: int, min_diff: int):
    genotypes_data_exp = genotypes_data.copy()
    
    for column in genotypes_data_exp.columns[5:]:
        split_data = genotypes_data_exp[column].str.split(":", expand=True)

        ids1 = split_data[2].astype(int) < min_experiments
        
        second_values = split_data[1].str.split(",").apply(lambda x: [int(i) for i in x] if isinstance(x, list) else [])
        
        second_largest = second_values.apply(lambda x: sorted(x)[-2] if len(x) > 1 else np.nan)
        ids2 = second_largest < min_diff

        genotypes_data_exp[column] = genotypes_data_exp[column].str.split(":").str[0]
        genotypes_data_exp.loc[ids1 | ids2, column] = "./."

    return genotypes_data_exp

In [25]:
time_data = pd.read_csv("Data.csv")
time_data = time_data[~time_data["value"].isna()]
time_data.head()

,Unnamed: 0,sample,year,value
14,14,PS000076,2015,114.0
15,15,PS000033,2015,116.0
16,16,PS000048,2015,99.0
17,17,PS000191,2015,71.0
18,18,PS000077,2015,90.0


In [26]:
import optuna

/home/hk04/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def objective(trial):
    #parameters 
    min_expirements_ = trial.suggest_int("min_experments", 5, 15, step=2) 
    min_diff_        = trial.suggest_int("min_diff", 10, 35, step=5) 
    count_dots_      = trial.suggest_int("count_dots", 4, 20, step=4) 

    gen_d_exp            = set_nans(min_experiments=min_expirements_, min_diff=min_diff_) 
    gen_to_merge         = gen_d_exp.iloc[:, 5:].T.reset_index().rename(columns={"index": "sample"})
    gen_to_merge.columns = gen_to_merge.columns.astype(str)

    count_dots   = gen_to_merge.apply(lambda col: (col == './.').sum())
    gen_to_merge = gen_to_merge.loc[:, count_dots <= count_dots_]  #count dots % of allowed NANs 

    final_dataset = pd.merge(pd.merge(time_data, gen_to_merge, on=["sample"]), weather_features, on=["year"])

    X, y = final_dataset.drop(columns=["value"]), final_dataset["value"]
    final_dataset.columns = final_dataset.columns.astype(str)
    cat_features          = np.array(gen_to_merge.columns[1:]).astype(str)
    X.columns             = X.columns.astype(str)

    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #Catboost
    model = CatBoostRegressor(iterations=1000,                                      #n iteratiojs
                            learning_rate=0.1,                                      #lr
                            depth=6,                                                #depth
                            cat_features=np.array(list(cat_features) + ["sample"]), #cat features
                            eval_metric='RMSE',                                     #metrics
                            verbose=False,                                          #echo is off
                            task_type='GPU')                                         
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)

    #metrics 
    r2 = r2_score(y_test, y_pred)

    return r2

study = optuna.create_study(direction="maximize") # maximise the score during tuning
study.optimize(objective, n_trials=100) # run the objective function 100 times

print(study.best_trial) # print the best performing pipeline

[I 2024-11-24 03:44:31,795] A new study created in memory with name: no-name-80d760c0-2548-4e35-a07d-247bfce26d41
[I 2024-11-24 03:47:34,869] Trial 0 finished with value: 0.1693877933329887 and parameters: {'min_experments': 7, 'min_diff': 15, 'count_dots': 4}. Best is trial 0 with value: 0.1693877933329887.
[I 2024-11-24 03:57:41,643] Trial 1 finished with value: 0.1918423193246923 and parameters: {'min_experments': 9, 'min_diff': 10, 'count_dots': 12}. Best is trial 1 with value: 0.1918423193246923.
[I 2024-11-24 04:06:20,753] Trial 2 finished with value: 0.10425674248323025 and parameters: {'min_experments': 11, 'min_diff': 30, 'count_dots': 16}. Best is trial 1 with value: 0.1918423193246923.
[I 2024-11-24 04:11:53,500] Trial 3 finished with value: 0.14943846480540057 and parameters: {'min_experments': 11, 'min_diff': 20, 'count_dots': 12}. Best is trial 1 with value: 0.1918423193246923.
[I 2024-11-24 04:13:50,073] Trial 4 finished with value: 0.10287459825804868 and parameters: {'